In [1]:
import datetime as dt
import time
import warnings
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.optimizers import Adam, Adamax, RMSprop
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [2]:
sns.set_style('whitegrid')
colors = sns.color_palette('deep', 8)
sns.set_palette(colors)

In [3]:
def prep_data(data, window_size = 1):
    data_reshaped = np.reshape(data, (data.shape[0], window_size, data.shape[1]))
    return data_reshaped

In [11]:
df_orig = pd.read_json("../datasets/df.json", orient='split')
df = df_orig.copy()
df = df[df.eth_close.isnull().sum():]
df.fillna(0, inplace=True)
df.set_index("date", drop=True, inplace=True)

# set time series data & target
X = df.eth_close
y = df.shift(-1).eth_close

# split into train/test
X_train, X_test = X[:-31], X[-31:-1]
y_train, y_test = y[:-31], y[-31:-1]

# reshape & scale
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

scaler = MinMaxScaler(feature_range=(0,1))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.fit_transform(y_test)

# prep for NN
X_train = prep_data(X_train)
X_test = prep_data(X_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1103, 1, 1) (30, 1, 1) (1103, 1) (30, 1)


In [22]:
def build_lstm(optimizer, lr):
    model = Sequential()
    model.add(LSTM(units=1,
                   activation='tanh',
                   use_bias=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [26]:
lstm = KerasRegressor(build_fn=build_lstm, verbose=0)

optimizers = ['Adam', 'Adamax', 'RMSprop']
learning_rates = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]

params = dict(optimizer=optimizers, lr=learning_rates)
grid = GridSearchCV(estimator=lstm, param_grid=params)
grid_results = grid.fit(X_train, y_train)

In [27]:
print(grid_results.best_params_)

{'lr': 0.006, 'optimizer': 'Adamax'}


In [47]:
print(grid_results.best_score_)

-0.0346309116501821


In [44]:
def validate(model, X, y):
    yhat = model.predict(X)
    yhat_invert = scaler.inverse_transform(yhat)
    y_orig = scaler.inverse_transform(y)
    rmse = np.sqrt(mean_squared_error(y_orig, yhat_invert))
    return (yhat_invert, y_orig, rmse)

In [48]:
lstm_ts = build_lstm('Adamax', 0.006)
lstm_ts.fit(X_train, y_train, epochs=500, verbose=1)

Epoch 1/500
1103/1103 [==============================] - 13s 12ms/step - loss: 0.0617
Epoch 2/500
1103/1103 [==============================] - 0s 117us/step - loss: 0.0566
Epoch 3/500
1103/1103 [==============================] - 0s 120us/step - loss: 0.0524
Epoch 4/500
1103/1103 [==============================] - 0s 122us/step - loss: 0.0486
Epoch 5/500
1103/1103 [==============================] - 0s 119us/step - loss: 0.0452
Epoch 6/500
1103/1103 [==============================] - 0s 118us/step - loss: 0.0422
Epoch 7/500
1103/1103 [==============================] - 0s 119us/step - loss: 0.0395
Epoch 8/500
1103/1103 [==============================] - 0s 118us/step - loss: 0.0373
Epoch 9/500
1103/1103 [==============================] - 0s 124us/step - loss: 0.0354
Epoch 10/500
1103/1103 [==============================] - 0s 120us/step - loss: 0.0338
Epoch 11/500
1103/1103 [==============================] - 0s 120us/step - loss: 0.0324
Epoch 12/500
1103/1103 [============================

1103/1103 [==============================] - 0s 135us/step - loss: 6.2915e-04
Epoch 183/500
1103/1103 [==============================] - 0s 137us/step - loss: 6.2793e-04
Epoch 184/500
1103/1103 [==============================] - 0s 138us/step - loss: 6.2692e-04
Epoch 185/500
1103/1103 [==============================] - 0s 147us/step - loss: 6.2615e-04
Epoch 186/500
1103/1103 [==============================] - 0s 135us/step - loss: 6.2638e-04
Epoch 187/500
1103/1103 [==============================] - 0s 211us/step - loss: 6.2485e-04
Epoch 188/500
1103/1103 [==============================] - 0s 134us/step - loss: 6.2399e-04
Epoch 189/500
1103/1103 [==============================] - 0s 128us/step - loss: 6.2369e-04
Epoch 190/500
1103/1103 [==============================] - 0s 121us/step - loss: 6.2256e-04
Epoch 191/500
1103/1103 [==============================] - 0s 133us/step - loss: 6.2172e-04
Epoch 192/500
1103/1103 [==============================] - 0s 126us/step - loss: 6.2104e-04
Ep

1103/1103 [==============================] - 0s 106us/step - loss: 5.4440e-04
Epoch 361/500
1103/1103 [==============================] - 0s 114us/step - loss: 5.4346e-04
Epoch 362/500
1103/1103 [==============================] - 0s 111us/step - loss: 5.4357e-04
Epoch 363/500
1103/1103 [==============================] - 0s 152us/step - loss: 5.4366e-04
Epoch 364/500
1103/1103 [==============================] - 0s 116us/step - loss: 5.4307e-04
Epoch 365/500
1103/1103 [==============================] - 0s 199us/step - loss: 5.4279e-04
Epoch 366/500
1103/1103 [==============================] - 0s 197us/step - loss: 5.4238e-04
Epoch 367/500
1103/1103 [==============================] - 0s 181us/step - loss: 5.4266e-04
Epoch 368/500
1103/1103 [==============================] - 0s 177us/step - loss: 5.4231e-04
Epoch 369/500
1103/1103 [==============================] - 0s 109us/step - loss: 5.4174e-04
Epoch 370/500
1103/1103 [==============================] - 0s 109us/step - loss: 5.4124e-04
Ep

In [49]:
train_val = validate(lstm_ts, X_train, y_train)
test_val = validate(lstm_ts, X_test, y_test)

print(train_val[2], test_val[2])

3.046798181223715 15.724197344907736


In [23]:
lstm = KerasRegressor(build_fn=build_lstm, verbose=0)

In [7]:
def fit_lstm(X_train, y_train, epochs=500, optimizer=Adam(lr=0.005),):
    model = Sequential()
    model.add(LSTM(units=1,
                   activation='tanh',
                   use_bias=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    return model

def validate(model, X, y):
    yhat = model.predict(X)
    yhat_invert = scaler.inverse_transform(yhat)
    y_orig = scaler.inverse_transform(y)
    rmse = np.sqrt(mean_squared_error(y_orig, yhat_invert))
    return (yhat_invert, y_orig, rmse)